In [1]:
import pickle
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from darts.models.forecasting.rnn_model import RNNModel
import sys
sys.path.append('/home/studio-lab-user/sybil')

from src.models.modelfactory import ModelFactory
from src.models.modelwrappers import DartsWrapper
from src.models.ts_utils import smape, mape

/home/studio-lab-user/.conda/envs/darts1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from darts.models import LightGBMModel
from darts import TimeSeries

In [3]:
dataset = "/home/studio-lab-user/sybil/datasets/climate/temp_anom_w_forcing.csv"

In [4]:
dataset = ModelFactory.prepare_dataset(pd.read_csv(dataset))
train_data, test_data = train_test_split(dataset, test_size=0.2, shuffle=False)



In [5]:
train_data

,WMGHG,Ozone,Solar,Land_Use,SnowAlb_BC,Orbital,TropAerDir,TropAerInd,StratAer,Anomaly
Year,,,,,,,,,,
1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.17
1851,0.005954,0.000745,0.001166,-0.000940,0.000649,0.000653,0.000649,-0.000005,0.029088,-0.09
1852,0.010502,0.000909,-0.010756,-0.002531,0.000649,0.000671,0.000649,-0.000005,0.056712,-0.11
1853,0.014587,0.001020,-0.027220,-0.004123,0.000649,0.000688,0.000649,-0.000005,0.077260,-0.12
1854,0.020521,0.001188,-0.045539,-0.005717,0.000649,0.000689,0.000649,-0.000005,0.085759,-0.12
...,...,...,...,...,...,...,...,...,...,...
1975,1.803290,0.314813,-0.034784,-0.194353,0.050885,0.001906,-0.099970,-0.498719,-0.668929,0.01
1976,1.854150,0.324083,-0.017348,-0.195037,0.051399,0.001918,-0.103195,-0.509644,-0.248317,-0.04
1977,1.913380,0.333346,0.011627,-0.195733,0.051905,0.001928,-0.106400,-0.523880,-0.035946,0.21


In [6]:
# Prepare the dataset and create the model (not using the mode factory here)
darts_model = RNNModel(model='LSTM', input_chunk_length=12, output_chunk_length=1, n_epochs=10)
#model = DartsWrapper(model=darts_model, type='darts_rnn', scorers=[smape, mape])

ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.


In [7]:
y_time_series = TimeSeries.from_series(train_data['Anomaly'])

In [8]:
lgbm_model = LightGBMModel(lags=12,)

In [9]:
lgbm_training_info = lgbm_model.fit(y_time_series)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000057 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 365
[LightGBM] [Info] Number of data points in the train set: 118, number of used features: 12
[LightGBM] [Info] Start training from score -0.118983
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [12]:
# exp

In [10]:
rnn_training_info = darts_model.fit(y_time_series)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 2.8 K 
4 | V             | Linear           | 26    
---------------------------------------------------
2.8 K     Trainable params
0         Non-trainable params
2.8 K     Total params
0.011     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 4/4 [00:00<00:00, 37.32it/s, train_loss=0.0263]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 4/4 [00:00<00:00, 36.07it/s, train_loss=0.0263]


In [11]:
lgbm_model.predict(5)

<TimeSeries (DataArray) (Year: 5, component: 1, sample: 1)>
array([[[ 0.08043495]],

       [[ 0.04060887]],

       [[ 0.04931382]],

       [[ 0.05031038]],

       [[-0.00735708]]])
Coordinates:
  * Year       (Year) int64 1980 1981 1982 1983 1984
  * component  (component) object 'Anomaly'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [12]:
darts_model.predict(5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 101.33it/s]


<TimeSeries (DataArray) (Year: 5, component: 1, sample: 1)>
array([[[-0.12728672]],

       [[-0.13615575]],

       [[-0.1421869 ]],

       [[-0.14618216]],

       [[-0.14881558]]])
Coordinates:
  * Year       (Year) int64 1980 1981 1982 1983 1984
  * component  (component) object 'Anomaly'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [13]:
type(rnn_training_info)

darts.models.forecasting.rnn_model.RNNModel

In [15]:
p = pickle.dumps(lgbm_training_info)
model_lgbm_load = pickle.loads(p)

In [16]:
model_lgbm_load.predict(5)

<TimeSeries (DataArray) (Year: 5, component: 1, sample: 1)>
array([[[ 0.08043495]],

       [[ 0.04060887]],

       [[ 0.04931382]],

       [[ 0.05031038]],

       [[-0.00735708]]])
Coordinates:
  * Year       (Year) int64 1980 1981 1982 1983 1984
  * component  (component) object 'Anomaly'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [18]:
p_rnn = pickle.dumps(rnn_training_info)
model_rnn_load = pickle.loads(p_rnn)

In [19]:
model_rnn_load

RNNModel(model=LSTM, hidden_dim=25, n_rnn_layers=1, dropout=0.0, training_length=24, input_chunk_length=12, output_chunk_length=1, n_epochs=10)

In [20]:
model_rnn_load.predict(5)

AttributeError: 'NoneType' object has no attribute 'set_predict_parameters'

In [21]:
rnn_training_info.save("rnn.pt")

In [22]:
new_model_load = RNNModel.load('rnn.pt')

In [23]:
new_model_load.predict(5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 138.62it/s]


<TimeSeries (DataArray) (Year: 5, component: 1, sample: 1)>
array([[[-0.12728672]],

       [[-0.13615575]],

       [[-0.1421869 ]],

       [[-0.14618216]],

       [[-0.14881558]]])
Coordinates:
  * Year       (Year) int64 1980 1981 1982 1983 1984
  * component  (component) object 'Anomaly'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None